In [2]:
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import scipy.optimize as opt

data2 = np.genfromtxt('/Users/bryanchia/Desktop/BootCamp2017/ProbSets/Econ/Wk4_StrEst/data/MacroSeries.txt', delimiter = ',')

c = data2[:,0]
k = data2[:,1]
w = data2[:,2]
r = data2[:,3]

Question 4
==============

In [3]:
def model_moments(c, k, w, r, alpha, beta, rho, mu):

    z = np.log(r) +(1 - alpha)*np.log(k) - np.log(alpha)
    model_mom1 = (z - rho*np.append(mu, z[:-1]) - (1 - rho)*mu).mean()
    model_mom2 = ((z - rho*np.append(mu, z[:-1]) - (1 - rho)*mu)*np.append(mu, z[:-1])).mean()
    model_mom3 = ((beta*alpha*np.exp(z[1:])*(k[1:]**(alpha - 1))*c[:-1]/c[1:]) - 1).mean()
    model_mom4 = (((beta*alpha*np.exp(z[1:])*(k[1:]**(alpha - 1))*c[:-1]/c[1:]) - 1)*w[:-1]).mean()
    
    return model_mom1, model_mom2, model_mom3, model_mom4

moms_data = np.array([[1e-30], [1e-30], [1e-30], [1e-30]])

def err_vec(c, k, w, r, alpha, beta, rho, mu, moms_data, simple = False):

    model_mom1, model_mom2, model_mom3, model_mom4 = model_moments(c, k, w, r, alpha, beta, rho, mu)
    moms_model = np.array([[model_mom1], [model_mom2], [model_mom3], [model_mom4]])
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data)/moms_model
    
    return err_vec

def crit(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, moms_data, W, simple = args
    err = err_vec(c, k, w, r, alpha, beta, rho, mu, moms_data, simple)
    crit_val = np.dot(np.dot(err.T, W), err)
    
    return crit_val

In [4]:
alpha_init = 0.5
beta_init = 0.5
rho_init = 0.5
mu_init = 9.5
bnds = ((1e-5, 1-(1e-5)), (1e-5, 1-(1e-5)), ((1e-5)-1, 1-(1e-5)), (1e-5, None))
params_init = (alpha_init, beta_init, rho_init, mu_init)
gmm_args = (c, k, w, r, moms_data, np.eye(4), True)
results = opt.minimize(crit, params_init, args=(gmm_args), method='L-BFGS-B', bounds=bnds)
results
alpha_MLE, beta_MLE, rho_MLE, mu_MLE  = results.x
print("MLE results: ", "\nRho = ", rho_MLE, "\nAlpha = ", alpha_MLE, "\nMu = ", mu_MLE, "\nBeta = ", beta_MLE)
print("Hessian = ", results.hess_inv.todense())

MLE results:  
Rho =  0.50539446811 
Alpha =  0.450454101541 
Mu =  9.49691995889 
Beta =  0.98999999502
Hessian =  [[  6.58972667e+07   3.96669088e+02  -7.17477566e+06   4.09653030e+06]
 [  3.96669088e+02   2.38775254e-03  -4.31886168e+01   2.46590951e+01]
 [ -7.17477566e+06  -4.31886168e+01   7.81177657e+05  -4.46022844e+05]
 [  4.09653030e+06   2.46590951e+01  -4.46022844e+05   2.54663471e+05]]
